In [25]:
from torchtext import data, datasets
import torch
import numpy  as np
from torch import nn, optim
import time
import torch.nn.functional as F


In [10]:
questions =  data.Field(tokenize = 'spacy', batch_first = True) # uses spacy for tokenization
labels = data.LabelField(dtype = torch.float) # sets as floats so we can caluclate gradients.

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [11]:
train_data, _ = datasets.TREC.splits(questions, labels)
train_data, valid_data = train_data.split()
print(len(train_data), len(valid_data))

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


3816 1636


In [12]:
# visualise sample
ith = np.random.randint(len(train_data))
print(train_data.examples[ith].text)
print(train_data.examples[ith].label)

['What', 'are', 'you', 'hearing', 'when', 'you', 'put', 'a', 'seashell', 'to', 'your', 'ear', '?']
DESC


In [13]:
questions.build_vocab(train_data, vectors="glove.6B.200d", unk_init=torch.Tensor.normal_)
labels.build_vocab(train_data)

In [14]:
questions.vocab.vectors

tensor([[ 0.6222, -0.0975, -0.3755,  ...,  1.8149,  1.1560,  0.9560],
        [-0.7594, -0.2627,  1.1070,  ...,  0.4254, -1.6362, -0.5985],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.6188, -0.0304, -0.5277,  ...,  0.0882, -0.2951, -0.2156],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570],
        [-0.2425,  0.6194,  1.3442,  ...,  0.4479,  1.3322, -0.3131]])

In [15]:
# create data iterators
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = 64,
    device = device
)


/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [27]:
class CNN(nn.Module):
    def __init__(self,
                 vocab_size, 
                 embedding_dim, 
                 n_filters, 
                 filter_sizes, 
                 output_dim, 
                 dropout, 
                 pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                      embedding_dim=embedding_dim, 
                                      padding_idx=pad_idx) # padding_index ->index of embeddign used to pad to same length, define later manually
        self.convs = nn.ModuleList( # define a list of convs layers
            [nn.Conv2d(in_channels=1, # input text does not have channels
                       out_channels=n_filters, # how many conv filters with same shape to train (same for all filter_sizes) 
                       kernel_size=(fs, embedding_dim)
                      ) 
             for fs in filter_sizes]
        )
        
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout= nn.Dropout(dropout)
        
        
    def forward(self, text):
        emb = self.embedding(text).unsqueeze(1)
        conved = [F.relu(conv2d(emb)).squeeze(3) for conv2d in self.convs]
        pooled = [F.max_pool1d(c , c.shape[2]).squeeze(2) for c in conved]
        concat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(concat)
    

In [28]:
input_dimension = len(questions.vocab)
output_dimension = 6
embedding_dimension = 200
pad_index = questions.vocab.stoi[questions.pad_token]
number_of_filters = 100
filter_sizes = [2, 3, 4]
dropout_pc = 0.5

model = CNN(vocab_size=input_dimension, 
                 embedding_dim = embedding_dimension, 
                 n_filters = number_of_filters, 
                 filter_sizes = filter_sizes, 
                 output_dim = output_dimension, 
                 dropout=dropout_pc, 
                 pad_idx=pad_index)

In [29]:
# load glove as embeddings  for tokens
glove_embeddings = questions.vocab.vectors
model.embedding.weight.data.copy_(glove_embeddings)

tensor([[ 0.6222, -0.0975, -0.3755,  ...,  1.8149,  1.1560,  0.9560],
        [-0.7594, -0.2627,  1.1070,  ...,  0.4254, -1.6362, -0.5985],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.6188, -0.0304, -0.5277,  ...,  0.0882, -0.2951, -0.2156],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570],
        [-0.2425,  0.6194,  1.3442,  ...,  0.4479,  1.3322, -0.3131]])

In [30]:
# handels unknown and paddings
unknown_index = questions.vocab.stoi[questions.unk_token]
model.embedding.weight.data[unknown_index] = torch.zeros(embedding_dimension)
model.embedding.weight.data[pad_index] = torch.zeros(embedding_dimension)


In [31]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)
model = model.to(device)

In [32]:
# trainig the CNN
def multi_accuracy(preds, y):
    pred = torch.max(preds, 1).indices
    correct = (pred == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [33]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        preds = model(batch.text).squeeze(1)
        loss = criterion(preds, batch.label.long())
        acc = multi_accuracy(preds, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    total_epoch_loss = epoch_loss/len(iterator)
    total_epoch_accuracy = epoch_acc/len(iterator)
        
    return total_epoch_loss, total_epoch_accuracy

In [38]:
def model_eval(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            preds = model(batch.text).squeeze(1)
            loss = criterion(preds, batch.label.long())
            acc = multi_accuracy(preds, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    total_epoch_loss = epoch_loss/len(iterator)
    total_epoch_accuracy = epoch_acc/len(iterator)
        
    return total_epoch_loss, total_epoch_accuracy

In [40]:
epochs = 10
lowest_validation_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = model_eval(model, valid_iterator, criterion)
    end_time = time.time()
    
    if valid_loss < lowest_validation_loss:
        lowest_validation_loss = valid_loss
        torch.save(model.state_dict(), 'cnn_model.pt')
        
    print(f"epoch {epoch}/{epochs}, time {int(end_time-start_time)}s")
    print(f"Train loss: {train_loss} train acc: {train_acc}")
    print(f"Valid loss: {valid_loss} Valid acc: {valid_acc}")

epoch 0/10, time 4
Train loss: 0.17327417396008968 train acc: 0.9547395835320155
Valid loss: 0.47200516210152554 Valid acc: 0.8239182692307693
epoch 1/10, time 4
Train loss: 0.12371736156443755 train acc: 0.9692708333333333
Valid loss: 0.4684221727343706 Valid acc: 0.8267895304239713
epoch 2/10, time 4
Train loss: 0.09398410860449076 train acc: 0.9802083333333333
Valid loss: 0.4662626769680243 Valid acc: 0.8350694454633273
epoch 3/10, time 4
Train loss: 0.06255863606929779 train acc: 0.9893229166666667
Valid loss: 0.46698962037379926 Valid acc: 0.8361378197486584
epoch 4/10, time 4
Train loss: 0.04782793417883416 train acc: 0.9908854166666666
Valid loss: 0.47145902709319043 Valid acc: 0.837606838116279
epoch 5/10, time 4
Train loss: 0.038404157664626834 train acc: 0.99375
Valid loss: 0.4744143503216597 Valid acc: 0.8412126073470483
epoch 6/10, time 4
Train loss: 0.028709084431951245 train acc: 0.99609375
Valid loss: 0.48557186413269776 Valid acc: 0.838074253155635
epoch 7/10, time 4
Tr